In [1]:
from scraper import Scraper
import pandas as pd
import httpx
from bs4 import BeautifulSoup
from random import random, randint
import time
import pandas as pd
from datetime import date

C:\Users\mfonseca\AppData\Local\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
s = Scraper()

In [3]:
cities = ['amsterdam']
#sites = ['pararius','funda','kamernet']
#sites = ['kamernet']
sites = ['pararius']
postTypes = ['Buy','Rent']

In [4]:
s.run(cities,sites,postTypes,10000)

Running Site: pararius
    Running post types: Buy
        Running city: amsterdam
            Running property type: appartement
            Page: 1
{'status_code': 200, 'headers': Headers([('date', 'Mon, 09 Dec 2024 14:35:24 GMT'), ('content-type', 'text/html; charset=UTF-8'), ('transfer-encoding', 'chunked'), ('connection', 'keep-alive'), ('vary', 'Accept-Encoding'), ('cache-control', 'max-age=0, must-revalidate, no-cache, no-store, private'), ('expires', 'Mon, 09 Dec 2024 14:35:24 GMT'), ('set-cookie', 'latest_search_locations=%5B%22amsterdam%22%5D; expires=Tue, 09-Dec-2025 14:35:23 GMT; Max-Age=31535999; path=/; domain=.www.pararius.nl; secure; samesite=lax'), ('set-cookie', 'user_redirect_intent=%7B%22account%22%3A%22%5C%2Fkoopwoningen%5C%2Famsterdam%5C%2Fappartement%22%7D; expires=Tue, 10-Dec-2024 14:35:24 GMT; Max-Age=86400; path=/; secure; httponly; samesite=lax'), ('set-cookie', 'ujt_id=07b39f57-1189-4768-88f6-7753e1f77ad3; expires=Mon, 09-Jun-2025 14:35:24 GMT; Max-Age=15724

KeyboardInterrupt: 

In [ ]:
s.properties

In [ ]:
update_df = history.set_index('url')[['first_scrape_date', 'last_scrape_date']].join(
    s.properties.set_index('url')['scrape_date'], 
    how='outer',
    rsuffix=''
)

In [ ]:
new_properties = s.properties[s.properties['url'].isin(update_df[update_df['first_scrape_date'].isna()].index)].copy()

In [ ]:
new_properties['first_scrape_date'] = new_properties['scrape_date']
new_properties['last_scrape_date'] = new_properties['scrape_date']
new_properties = new_properties.drop(columns=['scrape_date'])

In [ ]:
properties_to_update = update_df[((~update_df['first_scrape_date'].isna())&
                                  (~update_df['scrape_date'].isna()))]

In [ ]:
history = history.set_index('url')
history.loc[history.index.isin(properties_to_update.index),'last_scrape_date'] = properties_to_update['scrape_date']
history = history.reset_index()

In [ ]:
history = pd.concat([history, new_properties])

In [ ]:
history.to_csv('data/history_20241207.csv', index=False)

In [ ]:
history = pd.read_csv('data/history_20241207.csv')

In [ ]:
postcode_coord = pd.read_csv('data/postcode_coordinates.csv')

In [ ]:
history['latitude'] = history['post_code'].map(postcode_coord.set_index('post_code')['latitude'].to_dict())
history['longitude'] = history['post_code'].map(postcode_coord.set_index('post_code')['longitude'].to_dict())

In [ ]:
history = history.rename(columns={'postType': 'post_type', 'post_code': 'postcode'})

In [ ]:
history.to_csv('data/history_20241207.csv', index=False)

In [ ]:
history = history[
    ['source', 'first_scrape_date', 'last_scrape_date', 'url', 'post_type',
    'city', 'location', 'title', 'postcode', 'price', 'price_type', 'surface',
    'surface_unit', 'rooms', 'furnished', 'latitude', 'longitude']
]

In [ ]:
history = history[~history['price'].isna()]

In [ ]:
history['price'] = history['price'].astype(int)

In [ ]:
history.to_csv('data/history_20241207_nohead.csv', index=False, header=False)